In [ ]:
import cv2
import numpy as np

def apply_clahe(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Check if the image is loaded properly
    if image is None:
        print(f"Error: Unable to load the image from {image_path}")
        return None

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(image)

    return enhanced_image

# Example usage
image_path = r'C:\Users\MANI JAGAN\Downloads\extracted_images\Data\TCGA_FG_A60K_20040224\TCGA_FG_A60K_20040224_1.tif'
enhanced_image = apply_clahe(image_path)

# Check if the enhancement was successful before saving
if enhanced_image is not None:
    cv2.imwrite('enhanced_image.tif', enhanced_image)
else:
    print("Image processing failed. No file saved.")


In [ ]:
def normalize_image(image):
    return image / 255.0


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

datagen = ImageDataGenerator(
    rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, 
    zoom_range=0.2, horizontal_flip=True
)

# Assuming `enhanced_image` is grayscale with shape (256, 256)
# Adding both batch and channel dimensions to make the shape (1, 256, 256, 1)
enhanced_image = np.expand_dims(enhanced_image, axis=-1)  # Add channel dimension (256, 256) -> (256, 256, 1)
augmented_images = datagen.flow(np.expand_dims(enhanced_image, axis=0))  # Add batch dimension -> (1, 256, 256, 1)


In [ ]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.models import Model

def build_nested_unet(input_shape):
    inputs = Input(input_shape)
    # Downsampling path
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    # Intermediate layers with additional nested connections would go here
    
    # Upsampling path
    up1 = UpSampling2D(size=(2, 2))(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    
    # Output layer
    output = Conv2D(1, (1, 1), activation='sigmoid')(conv2)
    
    model = Model(inputs=inputs, outputs=output)
    return model

model = build_nested_unet((128, 128, 1))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from tensorflow.keras.layers import Multiply, Activation, Add

def attention_block(x, g, inter_channels):
    theta_x = Conv2D(inter_channels, (2, 2), strides=(2, 2))(x)
    phi_g = Conv2D(inter_channels, (1, 1), padding='same')(g)
    concat_xg = Add()([theta_x, phi_g])
    act_xg = Activation('relu')(concat_xg)
    psi = Conv2D(1, (1, 1), padding='same')(act_xg)
    psi = Activation('sigmoid')(psi)
    upsample_psi = UpSampling2D(size=(2, 2))(psi)
    return Multiply()([upsample_psi, x])


In [ ]:
import tensorflow.keras.backend as K

def dice_coefficient(y_true, y_pred):
    intersection = K.sum(y_true * y_pred)
    return (2. * intersection) / (K.sum(y_true) + K.sum(y_pred) + K.epsilon())

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coefficient])


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load dataset (for example, MNIST)
(train_data, train_labels), (val_data, val_labels) = mnist.load_data()

# Normalize the data
train_data = train_data.astype('float32') / 255.0  # Normalize to [0, 1]
val_data = val_data.astype('float32') / 255.0

# Check if the data is 3D (height, width) and expand dimensions to (height, width, 1)
if len(train_data.shape) == 3:
    train_data = np.expand_dims(train_data, axis=-1)  # Shape: (num_samples, 28, 28, 1)

if len(val_data.shape) == 3:
    val_data = np.expand_dims(val_data, axis=-1)  # Shape: (num_samples, 28, 28, 1)

# Resize images to (128, 128)
train_data_resized = np.array([tf.image.resize(image, (128, 128)).numpy() for image in train_data])
val_data_resized = np.array([tf.image.resize(image, (128, 128)).numpy() for image in val_data])

# If images are grayscale, ensure they have a channel dimension (already done above)
# The shape should now be (num_samples, 128, 128, 1)

# Convert labels to categorical (one-hot encoding)
train_labels = to_categorical(train_labels, num_classes=10)
val_labels = to_categorical(val_labels, num_classes=10)

# Now you can use train_data_resized and train_labels
history = model.fit(train_data_resized, train_labels, epochs=50, validation_data=(val_data_resized, val_labels))


In [ ]:
from fastapi import FastAPI, File, UploadFile
from tensorflow.keras.models import load_model
import cv2
import numpy as np

app = FastAPI()
model = load_model('best_model.h5', custom_objects={'dice_coefficient': dice_coefficient})

@app.post("/predict/")
async def predict(file: UploadFile = File(...)):
    image = await file.read()
    np_image = np.frombuffer(image, np.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_GRAYSCALE)
    
    img_resized = cv2.resize(img, (128, 128))
    img_normalized = img_resized / 255.0
    img_input = np.expand_dims(img_normalized, axis=(0, -1))
    
    pred_mask = model.predict(img_input)
    return {"mask": pred_mask.tolist()}


In [ ]:
pip install fastapi

In [10]:
import streamlit as st
import requests
from PIL import Image

st.title("Brain MRI Metastasis Segmentation")

uploaded_file = st.file_uploader("Choose an MRI image...", type=["jpg", "png", "tif"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded MRI.', use_column_width=True)
    
    # Send the image to the backend API
    files = {"file": uploaded_file.getvalue()}
    response = requests.post("http://127.0.0.1:8000/predict/", files=files)
    
    # Display the segmentation result
    mask = np.array(response.json()['mask'])
    st.image(mask, caption='Predicted Mask', use_column_width=True)
    print(mask)


2024-10-01 11:26:29.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 11:26:29.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 11:26:29.364 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 11:26:29.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 11:26:29.366 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-01 11:26:29.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.
